## Engage3 Technical Challenge


In [1]:
import pandas as pd
import numpy as np
import json
pd.options.display.max_columns = 50
pd.options.display.max_rows = 500

Importing the given files into dataframes: 

In [2]:
df_stores=pd.read_json("stores.json", orient='columns')
df_stores= df_stores.rename(columns={'Region':'Store_Region'})
df_stores.head(20)


,Banner,Store_Region,Store ID
0,Walmart,Northern California,66999
1,Trader Joes,Northern California,4698
2,Safeway,Northern California,39482
3,Whole Foods,Northern California,34957
4,Walmart,New York,12837
5,Trader Joes,New York,9487
6,Wegmans,New York,2938
7,Whole Foods,New York,50948
8,Walmart,Kansas,40593
9,Trader Joes,Kansas,29384


In [3]:
df_prices=pd.read_csv("prices.csv")
df_prices.head(10)

,Auditor ID,Date,Price,Store ID,UPC
0,234,10/18/17,24.95,66999,268588472
1,234,10/27/17,49.71,66999,475245085
2,234,10/20/17,25.75,66999,126967843
3,234,10/23/17,18.81,66999,708930835
4,234,10/23/17,33.32,66999,325885139
5,234,10/16/17,40.43,66999,509419735
6,234,10/22/17,35.10,66999,736362595
7,234,10/21/17,33.65,66999,652377244
8,234,10/16/17,22.30,66999,277394644
9,234,10/20/17,12.32,66999,996849471


In [4]:
df_prices.describe()

,Auditor ID,Price,Store ID,UPC
count,12315.000000,12315.000000,12315.000000,1.231500e+04
mean,437.792205,29.018340,40283.130735,5.078112e+08
std,415.075558,18.445903,27062.251308,2.876799e+08
min,63.000000,0.590000,2938.000000,1.187317e+07
25%,98.000000,13.290000,29382.000000,2.664258e+08
50%,304.000000,28.520000,39482.000000,5.072036e+08
75%,713.000000,44.190000,50495.000000,7.644589e+08
max,1326.000000,74.490000,98736.000000,9.991851e+08


In [5]:
df_auditors= pd.read_csv("auditors.csv")
df_auditors= df_auditors.rename(columns={'Region':'Auditor_Region'})
df_auditors.head(10)


,Auditor ID,First,Last,Auditor_Region
0,234,Sue,Smith,Northern California
1,536,Bob,Smith,Northern California
2,98,Jack,Smith,New York
3,203,Jill,Smith,New York
4,304,Jerry,Johnson,Texas
5,63,Randy,Johnson,Texas
6,1326,Mike,Johnson,Kansas
7,713,Dave,Johnson,Kansas


In [6]:
#merging auditor and store dataframes onto prices
df_master= pd.merge(df_prices, df_auditors, on='Auditor ID')

In [7]:
df_master.describe()

,Auditor ID,Price,Store ID,UPC
count,12315.000000,12315.000000,12315.000000,1.231500e+04
mean,437.792205,29.018340,40283.130735,5.078112e+08
std,415.075558,18.445903,27062.251308,2.876799e+08
min,63.000000,0.590000,2938.000000,1.187317e+07
25%,98.000000,13.290000,29382.000000,2.664258e+08
50%,304.000000,28.520000,39482.000000,5.072036e+08
75%,713.000000,44.190000,50495.000000,7.644589e+08
max,1326.000000,74.490000,98736.000000,9.991851e+08


In [8]:
df_master= pd.merge(df_master,df_stores, on='Store ID')

In [9]:
df_master.describe()

,Auditor ID,Price,Store ID,UPC
count,11511.000000,11511.000000,11511.000000,1.151100e+04
mean,445.483972,28.745817,39821.673356,5.082154e+08
std,426.147737,18.418878,27784.695527,2.878573e+08
min,63.000000,0.590000,2938.000000,1.187317e+07
25%,98.000000,12.990000,12837.000000,2.664258e+08
50%,304.000000,28.240000,39482.000000,5.084538e+08
75%,713.000000,43.990000,50495.000000,7.644589e+08
max,1326.000000,74.490000,98736.000000,9.991851e+08


Because of the inner join used in the .merge, we can see that the master dataframe is missing 804 price records that have a StoreID not found within the store ID dataset provided. We can get unique store ID records from both columns and use a set difference command to find the missing values.


In [10]:
price_stores= df_prices['Store ID'].unique()
official_stores= df_stores['Store ID'].unique()
np.setdiff1d(price_stores,official_stores)

array([38472, 60957], dtype=int64)

We can see that the store IDs 38472 and 60957 do not have Banner and Regional data information, so for the time being they will not be included in regional analysis.

In [11]:
df_master.head()

,Auditor ID,Date,Price,Store ID,UPC,First,Last,Auditor_Region,Banner,Store_Region
0,234,10/18/17,24.95,66999,268588472,Sue,Smith,Northern California,Walmart,Northern California
1,234,10/27/17,49.71,66999,475245085,Sue,Smith,Northern California,Walmart,Northern California
2,234,10/20/17,25.75,66999,126967843,Sue,Smith,Northern California,Walmart,Northern California
3,234,10/23/17,18.81,66999,708930835,Sue,Smith,Northern California,Walmart,Northern California
4,234,10/23/17,33.32,66999,325885139,Sue,Smith,Northern California,Walmart,Northern California


In [12]:
df_master['Auditor Name']= df_master['First']+" "+ df_master['Last']
#combining two columns for readability in master sheet only

In [13]:
df_master.head()

,Auditor ID,Date,Price,Store ID,UPC,First,Last,Auditor_Region,Banner,Store_Region,Auditor Name
0,234,10/18/17,24.95,66999,268588472,Sue,Smith,Northern California,Walmart,Northern California,Sue Smith
1,234,10/27/17,49.71,66999,475245085,Sue,Smith,Northern California,Walmart,Northern California,Sue Smith
2,234,10/20/17,25.75,66999,126967843,Sue,Smith,Northern California,Walmart,Northern California,Sue Smith
3,234,10/23/17,18.81,66999,708930835,Sue,Smith,Northern California,Walmart,Northern California,Sue Smith
4,234,10/23/17,33.32,66999,325885139,Sue,Smith,Northern California,Walmart,Northern California,Sue Smith


In [14]:
columns=[ 'UPC','Banner','Date', 'Price','Store ID','Store_Region','Auditor ID','Auditor_Region','Auditor Name']
#changing order of columns

In [15]:
df_master=df_master[columns]
df_master.reset_index(drop=True)
df_master.head(10)

,UPC,Banner,Date,Price,Store ID,Store_Region,Auditor ID,Auditor_Region,Auditor Name
0,268588472,Walmart,10/18/17,24.95,66999,Northern California,234,Northern California,Sue Smith
1,475245085,Walmart,10/27/17,49.71,66999,Northern California,234,Northern California,Sue Smith
2,126967843,Walmart,10/20/17,25.75,66999,Northern California,234,Northern California,Sue Smith
3,708930835,Walmart,10/23/17,18.81,66999,Northern California,234,Northern California,Sue Smith
4,325885139,Walmart,10/23/17,33.32,66999,Northern California,234,Northern California,Sue Smith
5,509419735,Walmart,10/16/17,40.43,66999,Northern California,234,Northern California,Sue Smith
6,736362595,Walmart,10/22/17,35.10,66999,Northern California,234,Northern California,Sue Smith
7,652377244,Walmart,10/21/17,33.65,66999,Northern California,234,Northern California,Sue Smith
8,277394644,Walmart,10/16/17,22.30,66999,Northern California,234,Northern California,Sue Smith
9,996849471,Walmart,10/20/17,12.32,66999,Northern California,234,Northern California,Sue Smith


In [16]:
df_pivot=df_master
df_pivot.UPC= df_master.UPC.astype(str)
df_pivot['pivotid']= df_pivot['UPC'] +" " +df_pivot['Banner']
df_pivot.head()
#using the pivot function to cross tabulate regional prices alongside each other; creating pivotid to use as index

,UPC,Banner,Date,Price,Store ID,Store_Region,Auditor ID,Auditor_Region,Auditor Name,pivotid
0,268588472,Walmart,10/18/17,24.95,66999,Northern California,234,Northern California,Sue Smith,268588472 Walmart
1,475245085,Walmart,10/27/17,49.71,66999,Northern California,234,Northern California,Sue Smith,475245085 Walmart
2,126967843,Walmart,10/20/17,25.75,66999,Northern California,234,Northern California,Sue Smith,126967843 Walmart
3,708930835,Walmart,10/23/17,18.81,66999,Northern California,234,Northern California,Sue Smith,708930835 Walmart
4,325885139,Walmart,10/23/17,33.32,66999,Northern California,234,Northern California,Sue Smith,325885139 Walmart


In [17]:
dfpivot= pd.pivot_table(df_master, index='pivotid', columns=['Store_Region'], values='Price')

In [18]:
dfpivot["median_price"]= dfpivot.median(axis=1)
#adding median price

In [19]:
dfpivot.head(10)

Store_Region,Kansas,New York,Northern California,Texas,median_price
pivotid,,,,,
101201265 Safeway,47.99,NaN,56.29,48.49,48.49
101201265 Trader Joes,NaN,NaN,NaN,46.09,46.09
101201265 Walmart,NaN,45.15,NaN,43.71,44.43
101201265 Wegmans,NaN,48.49,NaN,47.09,47.79
101201265 Whole Foods,1.99,52.89,59.59,51.29,52.09
105664025 Safeway,22.49,NaN,NaN,NaN,22.49
105664025 Trader Joes,NaN,22.29,NaN,21.69,21.99
105664025 Walmart,NaN,21.26,23.90,20.54,21.26
105664025 Wegmans,NaN,22.79,NaN,22.09,22.44


In [20]:
dfpivot.reset_index(inplace=True)
dfpivot.head(10)

Store_Region,pivotid,Kansas,New York,Northern California,Texas,median_price
0,101201265 Safeway,47.99,NaN,56.29,48.49,48.49
1,101201265 Trader Joes,NaN,NaN,NaN,46.09,46.09
2,101201265 Walmart,NaN,45.15,NaN,43.71,44.43
3,101201265 Wegmans,NaN,48.49,NaN,47.09,47.79
4,101201265 Whole Foods,1.99,52.89,59.59,51.29,52.09
5,105664025 Safeway,22.49,NaN,NaN,NaN,22.49
6,105664025 Trader Joes,NaN,22.29,NaN,21.69,21.99
7,105664025 Walmart,NaN,21.26,23.90,20.54,21.26
8,105664025 Wegmans,NaN,22.79,NaN,22.09,22.44
9,105664025 Whole Foods,1.99,24.79,27.99,NaN,24.79


In [21]:
data= dfpivot["pivotid"].str.split(" ",n= 1, expand= True)
dfpivot["UPC"]= data[0]
dfpivot["Banner"]=data[1]
#returning UPC and Banner to different columns

In [22]:
dfpivot.reset_index(inplace=True)
data= dfpivot["pivotid"].str.split(" ",n= 1, expand= True)
dfpivot["UPC"]= data[0]
dfpivot["Banner"]=data[1]
columns2=[ 'UPC','Banner','Kansas','New York','Northern California','Texas', 'median_price']
dfpivot=dfpivot[columns2]
dfpivot.head()

Store_Region,UPC,Banner,Kansas,New York,Northern California,Texas,median_price
0,101201265,Safeway,47.99,NaN,56.29,48.49,48.49
1,101201265,Trader Joes,NaN,NaN,NaN,46.09,46.09
2,101201265,Walmart,NaN,45.15,NaN,43.71,44.43
3,101201265,Wegmans,NaN,48.49,NaN,47.09,47.79
4,101201265,Whole Foods,1.99,52.89,59.59,51.29,52.09


#### Exporting dataframes to excel

In [23]:

writer= pd.ExcelWriter('engage3_challenge.xlsx', engine='xlsxwriter')

dfpivot.to_excel(writer,sheet_name='Regional Data')
df_master.to_excel(writer,sheet_name='Master')
df_stores.to_excel(writer,sheet_name='Stores')
df_prices.to_excel(writer, sheet_name='Raw Price Data')
df_auditors.to_excel(writer,sheet_name='Auditors')
writer.save()

## Checking for Outliers/Anomalies

I'm using .loc function to pull records that are significantly below or above the median price of an item from a specific brand. 

In [24]:
dfpivot.loc[(dfpivot['Kansas'] < 0.5* dfpivot['median_price']) | (dfpivot['Kansas'] > 1.5* dfpivot['median_price'])].head(10)

Store_Region,UPC,Banner,Kansas,New York,Northern California,Texas,median_price
4,101201265,Whole Foods,1.99,52.89,59.59,51.29,52.09
9,105664025,Whole Foods,1.99,24.79,27.99,NaN,24.79
14,106365705,Whole Foods,1.99,22.49,25.29,21.79,22.14
19,106599165,Whole Foods,1.99,40.69,NaN,NaN,21.34
24,106918220,Whole Foods,1.99,25.89,29.19,25.09,25.49
29,107865183,Whole Foods,1.99,35.59,NaN,34.59,34.59
34,109076038,Whole Foods,1.99,38.89,NaN,37.79,37.79
39,110354720,Whole Foods,1.99,17.19,NaN,NaN,9.59
44,110497121,Whole Foods,1.99,NaN,27.39,23.59,23.59
49,111650456,Whole Foods,1.99,NaN,69.79,NaN,35.89


This analysis points out a significant discrepancy with the Whole Foods brand from the Kansas region, as many records, likely mistakenly, are listed at $1.99.

Further analysis from the dataframe shows there are 912 records of $1.99 records all submitted by AuditorID 713, Dave Johnson, all for the same Kansas based Whole Foods.

Analysis of the other regions does not find any glaring anomalies. 

In [25]:
dfpivot.loc[(dfpivot['New York'] < 0.5* dfpivot['median_price']) | (dfpivot['New York'] > 1.5* dfpivot['median_price'])].head(10)

Store_Region,UPC,Banner,Kansas,New York,Northern California,Texas,median_price
19,106599165,Whole Foods,1.99,40.69,NaN,NaN,21.34
39,110354720,Whole Foods,1.99,17.19,NaN,NaN,9.59
59,112817865,Whole Foods,1.99,61.59,NaN,NaN,31.79
117,124052151,Whole Foods,1.99,21.99,NaN,NaN,11.99
147,127887048,Whole Foods,1.99,9.69,NaN,NaN,5.84
162,130186387,Whole Foods,1.99,22.19,NaN,NaN,12.09
191,136528713,Whole Foods,1.99,15.39,NaN,NaN,8.69
211,144644625,Whole Foods,1.99,61.69,NaN,NaN,31.84
231,151145905,Whole Foods,1.99,29.69,NaN,NaN,15.84
236,151287985,Whole Foods,1.99,22.29,NaN,NaN,12.14


In [26]:
dfpivot.loc[(dfpivot['Northern California'] < 0.5* dfpivot['median_price']) | (dfpivot['Northern California'] > 1.5* dfpivot['median_price'])].head(10)

Store_Region,UPC,Banner,Kansas,New York,Northern California,Texas,median_price
49,111650456,Whole Foods,1.99,NaN,69.79,NaN,35.89
127,125829867,Whole Foods,1.99,NaN,17.89,NaN,9.94
157,129833275,Whole Foods,1.99,NaN,33.69,NaN,17.84
394,176351627,Whole Foods,1.99,NaN,66.89,NaN,34.44
428,189565540,Whole Foods,1.99,NaN,53.89,NaN,27.94
879,269567447,Whole Foods,1.99,NaN,72.79,NaN,37.39
993,286906735,Safeway,10.99,NaN,16.89,11.09,11.09
1134,311149036,Whole Foods,1.99,NaN,61.49,NaN,31.74
1380,361981249,Whole Foods,1.99,NaN,25.59,NaN,13.79
1385,363196884,Whole Foods,1.99,NaN,6.59,NaN,4.29


In [27]:
dfpivot.loc[(dfpivot['Texas'] < 0.5* dfpivot['median_price']) | (dfpivot['Texas'] > 1.5* dfpivot['median_price'])].head(10)

Store_Region,UPC,Banner,Kansas,New York,Northern California,Texas,median_price
398,177925497,Whole Foods,1.99,NaN,NaN,50.29,26.14
734,244899396,Whole Foods,1.99,NaN,NaN,37.39,19.69
759,248544810,Whole Foods,1.99,NaN,NaN,35.29,18.64
814,256906423,Whole Foods,1.99,NaN,NaN,10.69,6.34
1370,361415663,Whole Foods,1.99,NaN,NaN,28.39,15.19
1493,377365657,Whole Foods,1.99,NaN,NaN,62.49,32.24
1732,433536637,Whole Foods,1.99,NaN,NaN,41.89,21.94
2007,476935693,Whole Foods,1.99,NaN,NaN,34.39,18.19
2379,54236183,Whole Foods,1.99,NaN,NaN,23.79,12.89
2419,549109328,Whole Foods,1.99,NaN,NaN,26.29,14.14
